<img src="images/ASF_Blue.png" width="175" alt="ASF Logo" align="right">

# Creating a Mosaic from OPERA RTC-S1 Images

In this notebook, we will work with [OPERA Level-2 Radiometric Terrain Corrected (RTC)](https://www.jpl.nasa.gov/go/opera/products/rtc-product) Synthetic Aperture Radar (SAR) Backscatter products generated from Sentinel-1 (S1) SAR imagery. These products are processed by the [Observational Products for End-Users from Remote Sensing Analysis (OPERA)](https://www.jpl.nasa.gov/go/opera) project at the [Jet Propulsion Labratory (JPL)](https://www.jpl.nasa.gov/).

The RTC-S1 products use a Digital Elevation Model (DEM) to correct for the distortions inherent to SAR data due to the side-looking geometry used to acquire the imagery. The OPERA project uses the [Copernicus Global 30-m (GLO-30)](https://spacedata.copernicus.eu/collections/copernicus-digital-elevation-model) DEM to produce the RTC-S1 products.

OPERA RTC-S1 products are archived in [NASA's Earthdata Cloud (EDC)](https://www.earthdata.nasa.gov/technology/cloud-computing) by the [Alaska Satellite Facility (ASF)](https://asf.alaska.edu/). They are archived as [Cloud-Optimized GeoTIFFs (COGs)](https://www.cogeo.org/), which allows users to work with the data directly in the cloud and generate derived products without having to download the source rasters first. They can be accessed using ASF's Data Search tools via the [Vertex web portal](https://search.asf.alaska.edu/#/?zoom=3.000&center=-97.494,39.673&dataset=OPERA-S1&productTypes=RTC), [the ASF Search API]( https://docs.asf.alaska.edu/api/basics/) or using the [asf_search python package](https://docs.asf.alaska.edu/asf_search/basics/).

If you do not already have one, [please create a free Earthdata Login Account](https://urs.earthdata.nasa.gov/users/new) to access products using ASF Search Tools.

In this notebook, we will use the asf_search python package to search for OPERA RTC-S1 images available over an area and time period of interest, then we will mosaic them together into a single GeoTIFF file that can be saved to your compute environment.

## 0. Using this notebook

Please be patient when launching Binder. This is a free service, and it can take a while to start the environment, especially when many people are trying to access it at the same time.

Click in a code cell and click the Run button to execute the block of code. Make sure to run code blocks consecutively, as it each block contains parameters and imports required to run following cells.

An asterisk [*] displays next to the code block when it is actively running. It is replaced with a number, indicating the order in which the code cells were run, when the process is complete.

The python kernel can become stale if the notebook is left idle for too long. If this happens, it's best to reload the binder from the URL link provided. In many cases, simply refreshing the browser will not solve the problem.

## 1. Enter Earthdata Login credentials
[Earthdata credentials](https://urs.earthdata.nasa.gov/users/new) are required to access OPERA datasets. One way do this is to store your username and password in a `temp_netrc` file. We first explicitly set gdal environment variables necessary for accessing OPERA products. You will then be prompted for your Earthdata username and password, which will be stored in a `temp_netrc` file in the Binder environment.

In [ ]:
import os
from getpass import getpass

os.environ['GDAL_HTTP_COOKIEFILE'] = '/tmp/cookies.txt'
os.environ['GDAL_HTTP_COOKIEJAR'] = '/tmp/cookies.txt'
os.environ['GDAL_HTTP_NETRC'] = 'YES'
os.environ['GDAL_HTTP_NETRC_FILE'] = 'temp_netrc'

username = getpass('NASA Earthdata Login username: ')
password = getpass('NASA Earthdata Login password: ')

with open(os.environ['GDAL_HTTP_NETRC_FILE'], 'w') as f:
    f.write(f'machine urs.earthdata.nasa.gov login {username} password {password}')

## 2. Search for OPERA RTC products over San Francisco
Search for OPERA RTC-S1 Products over San Francisco

We will use the asf_search python package to search for OPERA RTC-S1 products over San Francisco, California. The RTC-S1 products are generated from indiviual Sentinel-1 SLC bursts, so they have a much smaller footprint than we are accustomed to seeing when working with Sentinel-1 data. Most users work with Sentinel-1 Level 1 products, including GRD and SLC IW products, which package together many individual bursts into larger scenes. Refer to [ASF's Sentinel-1 Burst Tutorial](https://storymaps.arcgis.com/stories/88c8fe67933340779eddef212d76b8b8) for more information on this topic.

For example, if we were to search for Sentinel-1 SLC products in Vertex for an AOI in the San Francisco area, we would see a few footprints with very large spatial coverage.

<img src="images/Vertex_Search_SLC.PNG" alt="Vertex search for SLC produccts over San Francisco, CA." align='center'>

If we do the same search for OPERA RTC-S1 products, there are more footprints, but each covers a much smaller area, and there is not as much unnecessary coverage beyond our area of interest.

<img src="images/Vertex_Search_RTC-S1.PNG" alt="Vertex search for OPERA RTC-S1 produccts over San Francisco, CA." align='center'>

It is easy to select the individual bursts of interest and mosaic them together to provide just the coverage we need. This is especially useful when the area of interest falls on the boundary between two of the SLC IW frames.

In [ ]:
import asf_search

options = {
	'intersectsWith': 'POLYGON((-122.5306 37.6278,-122.1372 37.6207,-122.1372 37.9566,-122.5287 37.9554,-122.5306 37.6278))',
	'dataset': 'OPERA-S1',
	'start': '2023-11-14T14:00:00Z',
	'end': '2023-11-14T15:00:00Z',
	'flightDirection': 'DESCENDING',
	'processingLevel': 'RTC',
	'maxResults': '1000'
}

results = asf_search.search(**options)
print(f'Found {len(results)} products that match your search criteria')

## 3. Get URLs for selected bursts from search results
The OPERA RTC-S1 products are archived in NASA's Earthdata Cloud, and can be each be accessed by a specific product URL. These URLs can be used to download the data to your local computer, but can also be used to perform cloud-based analyses that access the pixel values of the archived source rasters without actually downloading them or moving them to a different location.

The asf_search results include the products URLs, which we will list for use in the next processing step.

In [ ]:
polarization = 'VV'

urls = []
for result in results:
	for url in result.umm['RelatedUrls']:
		if url['URL'].startswith('https://') and url['URL'].endswith(f'{polarization}.tif'):
			urls.append('/vsicurl/' + url['URL'])

## 4. Mosaic bursts into a single GeoTIFF and save

We will use the gdal_merge command from the osgeo_utils python package for the [GDAL geospatial data format translator library](https://gdal.org/) to mosaic bursts together into a single GeoTIFF, which we will save to memory in the Binder environment.

In [ ]:
from osgeo import gdal
import osgeo_utils.gdal_merge as gdal_merge

gdal.DontUseExceptions()
output_file_path = f'{os.getcwd()}/OPERA_RTC_San_Francisco.tif'

parameters = ['', '-a_nodata', 'nan', '-o', output_file_path] + urls
gdal_merge.main(parameters)

## 5. Plot merged GeoTIFF

Let's take a look at the output product! We'll set a min-max grayscale stretch that displays the main range of the backscatter values in this area. The RTC-S1 values use the power scale, so many of the values are very close to 0.

We set the maximum value for the stretch to be 0.25, but you can experiment with different min-max values to focus more closely on specific ranges of backscatter values.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import rasterio
import numpy as np

src = rasterio.open(output_file_path)
arr = src.read(1)
masked_array = np.ma.array(arr, mask=(arr==0.0))

f, ax = plt.subplots(1,1, figsize=(10,10))

im = ax.imshow(masked_array, cmap='grey', vmin=0, vmax=0.25)
ax.set_title('Merged OPERA RTC-S1 VV Products', fontsize=16)
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])

cbar = plt.colorbar(im, shrink=0.6, ticks=[0, 0.1, 0.2])
cbar.set_label('Backscatter (Power)', fontsize=12)
cbar.ax.set_yticklabels(['0', '0.1', '0.2'])
cbar.ax.yaxis.set_label_position('left')

plt.tight_layout()

## 6. Save merged GeoTIFF to local workspace (OPTIONAL)
You can save the merged geotiff to your local space using the Jupyter notebook GUI.  In the project panel on the left (expand it if necessary), right-click on the OPERA_RTC_San_Francisco.tif file, and select the 'Download' option.

<img src="images/save_geotiff_locally.png" alt="Screenshot showing how to download the merged geotiff to your localy workspace." align='center'>